<h2><center>CUDA PROGRAMMING</center></h2>
First component of a typical CUDA program is the kernel program which is marked with a global keyword.  

In [ ]:
#Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      y[i] = x[i] + y[i];
}

This function is executed by an array of CUDA threads called thread blocks and each thread has an id with which it uses memory adresses and makes informed decisions. These 1000's of threads can be executed in GPU and CUDA organizes these threads into Grids heirarchy of thread blocks.<br>

![title](img/cuda_indexing.png)

A Grid is a set of thread blocks that can be processed on the device in parallel. A block is a set of concurrent threads that can cooperate among themselves and access a shared memory space private to the block.<br>

![title](img/Block-thread.svg.png)

<h4>How CUDA GPU processes a 512x512 image </h4>

Suppose we want one thread to process one pixel (i,j)<br>
1. We can use blocks of 64 threads each. A block is executed by a single multiprocessing unit. Then we need 512*512/64 = 4096 blocks (so to have 512x512 threads = 4096*64).<br>
2. It's common to organize (to make indexing the image easier) the threads in 2D blocks having blockDim = 8 x 8 (the 64 threads per block)
3. Set grid dimension as 2D gridDim = 64 x 64 blocks (the 4096 blocks needed)
4. Then the kernel is launched as myKernel <<<numBlocks,threadsPerBlock>>>
5. Finally: there will be something like "a queue of 4096 blocks", where a block is waiting to be assigned one of the multiprocessors of the GPU to get its 64 threads executed

<h8>Source : https://stackoverflow.com/questions/2392250/understanding-cuda-grid-dimensions-block-dimensions-and-threads-organization-s</h8>

<h4>A simple Cpp program in C++ to add arrays</h4>
Source : 

In [ ]:
#include <iostream>
#include <math.h>
#include <ctime>
using namespace std;

//function to add two arrays
void add(int N, float *x, float *y)
{
  for(int i = 0; i < N; i++)
  {
    y[i] = x[i] + y[i];
  }
}

int main(void)
{
  clock_t begin_time = clock(); //set begin time

  int N = 1<<20; // 1 Million elements

  float *x = new float[N]; //pointer to the big array
  float *y = new float[N];
  /*A pointer is a variable whose value is the address of another variable*/

  //initialize these arrays with values
  for(int i = 0; i < N; i++)
  {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  //add the two variables together
  add(N, x, y);

  //Free memory
  delete [] x;
  delete [] y;
  clock_t end_time = clock();
  float diff ((float)end_time - (float)begin_time);
  float seconds = diff/CLOCKS_PER_SEC;
  cout << "time taken" <<endl;
  cout << end_time;
  cout << seconds << endl;
  return 0;

}


<h4>A simple CUDA program in C++ to add arrays</h4>
Source : https://github.com/llSourcell/An_Introduction_to_GPU_Programming/blob/master/add.cu

In [ ]:
#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  int index = threadIdx.x;
  int stride = blockDim.x;
  for (int i = index; i < n; i += stride)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 256>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);
  
  return 0;
}